# Import libraries

In [3]:
import numpy as np, pandas as pd, msprime, tskit, pathlib

In [4]:
# ------- CONFIG -------
rng = np.random.default_rng(42)
contig_length = 50_000_000             # 50 Mb window
mu = 1.25e-8                           # per-base per-gen mutation rate
N_anc = 12_000                         # ancestral Ne
split_time = 2_000                     # generations ago
m12 = m21 = 5e-4                       # symmetric migration
n_A = 500                              # samples in pop A
n_B = 500                              # samples in pop B
recomb_rate = 1e-8                     # fallback if not using a map
# Optional: use an external recombination map if you have a file:
recomb_map = None      

In [5]:
# ------- DEMOGRAPHY -------
dem = msprime.Demography()
dem.add_population(name="ancestral", initial_size=N_anc)
dem.add_population(name="A", initial_size=N_anc)
dem.add_population(name="B", initial_size=N_anc)
dem.add_population_split(time=split_time, derived=["A","B"], ancestral="ancestral")
dem.set_migration_rate("A", "B", m12)
dem.set_migration_rate("B", "A", m21)

In [6]:
# ------- ANCESTRY -------
if recomb_map is None:
    ts_anc = msprime.sim_ancestry(
        samples={"A": n_A, "B": n_B},
        sequence_length=contig_length,
        recombination_rate=recomb_rate,
        demography=dem,
        random_seed=12345,
    )
else:
    ts_anc = msprime.sim_ancestry(
        samples={"A": n_A, "B": n_B},
        recombination_rate=recomb_map,
        demography=dem,
        random_seed=12345,
    )


In [7]:
# ------- MUTATIONS -------
ts = msprime.sim_mutations(ts_anc, rate=mu, discrete_genome=True, random_seed=54321)

# ------- OUTPUT: variant table + genotypes -------
# Per-site metadata
sites = []
for v in ts.variants():
    sites.append({"pos": int(v.site.position), "id": f"sim_{v.site.id}", "alleles": v.alleles})
sites = pd.DataFrame(sites).sort_values("pos").reset_index(drop=True)

In [8]:
# Sample table: map samples to populations
sample_pops = []
for i, n in enumerate(ts.samples()):
    pop_id = ts.node(n).population
    pop_name = ts.population(pop_id).metadata.get("name") if ts.population(pop_id).metadata else None
    sample_pops.append({"sample_index": i, "node": n, "population_id": pop_id,
                        "population": pop_name if pop_name else ("A" if pop_id==1 else "B")})
sample_df = pd.DataFrame(sample_pops)

# Dense genotype matrix (variants x samples) as 0/1/2
G = np.vstack([v.genotypes for v in ts.variants()])   # shape: n_variants x n_samples
# ts.variants() yields haploid calls per sample for haploid samples; here samples are diploids by default in msprime>=1.3 if ploidy=2.
# To ensure diploid genotypes, pass ploidy=2 in sim_ancestry and reshape; or keep ploidy=1 and later combine pairs as needed.


In [13]:
# Save to your pipeline’s expected format (e.g., parquet)
outdir = pathlib.Path("simulation_realistic/msprime_A_B")
outdir.mkdir(parents=True, exist_ok=True)
sites.to_parquet(outdir / "sites.parquet", index=False)
sample_df.to_parquet(outdir / "samples.parquet", index=False)
pd.DataFrame(G).to_parquet(outdir / "genotypes_raw.parquet", index=False)

In [14]:
import numpy as np, pandas as pd
from pathlib import Path

def maf(g_col):
    # g_col: 0/1/2 diploid genotype vector (n_samples,)
    p = np.mean(g_col)/2.0
    return min(p, 1 - p)

def ld_prune(geno_df, positions, r2_thresh=0.2, window=50_000, step=5_000):
    # Very simple windowed pruning by pairwise r^2; replace with faster LDkit if needed.
    keep = []
    pos = positions.values
    G = geno_df.values  # variants x samples
    last_kept_idx = -np.inf
    for i in range(G.shape[0]):
        if pos[i] < last_kept_idx + step:
            continue
        # compare to kept variants within window
        ok = True
        j_idx = [j for j in keep if pos[i]-pos[j] <= window]
        for j in j_idx:
            # r^2
            gi = G[i]; gj = G[j]
            ci = gi - gi.mean(); cj = gj - gj.mean()
            r = (ci @ cj) / np.sqrt((ci@ci)*(cj@cj) + 1e-12)
            if r*r > r2_thresh:
                ok = False; break
        if ok:
            keep.append(i)
            last_kept_idx = pos[i]
    return np.array(keep, dtype=int)

def ascertain_chip(genotypes_df, sites_df, samples_df,
                   discovery_pop="A", discovery_n=200,
                   maf_range=(0.05, 0.5), n_snps_target=600_000,
                   ld_r2=0.2, ld_window=50_000, ld_step=5_000,
                   rng=np.random.default_rng(1)):
    """
    - Choose a discovery panel in 'discovery_pop'
    - Keep variants with MAF in [maf_range]
    - LD prune to approximate array independence
    - Downsample to n_snps_target if needed
    - Return filtered tables and an index vector
    """
    # Identify discovery samples
    disc_idx = samples_df.query("population == @discovery_pop")["sample_index"].to_numpy()
    G = genotypes_df.values  # variants x samples

    # Compute MAFs in discovery
    maf_vals = np.apply_along_axis(lambda g: maf(g[disc_idx]), axis=1, arr=G)

    keep_maf = (maf_vals >= maf_range[0]) & (maf_vals <= maf_range[1])
    G_maf = G[keep_maf]
    sites_maf = sites_df.loc[keep_maf].reset_index(drop=True)

    # LD prune
    keep_idx_local = ld_prune(pd.DataFrame(G_maf), sites_maf["pos"], r2_thresh=ld_r2, window=ld_window, step=ld_step)
    G_pruned = G_maf[keep_idx_local]
    sites_pruned = sites_maf.iloc[keep_idx_local].reset_index(drop=True)

    # Downsample to array size
    if G_pruned.shape[0] > n_snps_target:
        sel = rng.choice(G_pruned.shape[0], size=n_snps_target, replace=False)
        sel.sort()
        G_chip = G_pruned[sel]
        sites_chip = sites_pruned.iloc[sel].reset_index(drop=True)
        idx_chip = np.flatnonzero(keep_maf)[keep_idx_local][sel]
    else:
        G_chip, sites_chip = G_pruned, sites_pruned
        idx_chip = np.flatnonzero(keep_maf)[keep_idx_local]

    return (pd.DataFrame(G_chip), sites_chip, idx_chip)

# ----- use it -----
geno = pd.read_parquet("simulation_realistic/msprime_A_B/genotypes_raw.parquet")   # variants x samples, 0/1/2
sites = pd.read_parquet("simulation_realistic/msprime_A_B/sites.parquet")
samples = pd.read_parquet("simulation_realistic/msprime_A_B/samples.parquet")

G_chip, sites_chip, idx = ascertain_chip(
    genotypes_df=geno, sites_df=sites, samples_df=samples,
    discovery_pop="A", discovery_n=200,
    maf_range=(0.05, 0.5), n_snps_target=600_000,
    ld_r2=0.2, ld_window=50_000, ld_step=5_000
)



In [ ]:
import pandas as pd
from pathlib import Path
from ascertainment_diploid import ascertain_chip_diploid, AscertainmentConfig

# Load your current files (haplotype-coded 0/1 matrix)
geno_hap = pd.read_parquet("simulation_realistic/msprime_A_B/genotypes_raw.parquet")  # variants x haplotypes
sites    = pd.read_parquet("simulation_realistic/msprime_A_B/sites.parquet")         # must have 'pos' (bp)
samples  = pd.read_parquet("simulation_realistic/msprime_A_B/samples.parquet")       # hap rows; include 'population'

cfg = AscertainmentConfig(
    discovery_pop="A",   # the population you want to mimic array ascertainment on
    maf_min=0.05, maf_max=0.50,
    n_snps_target=600_000,
    ld_r2=0.2, ld_window_bp=50_000, ld_step_bp=5_000,
    random_seed=1
)

res = ascertain_chip_diploid(
    genotypes_hap_df=geno_hap,
    sites_df=sites,
    samples_df=samples,
    config=cfg,
    pairing=None,  # set to list of (hap1_col, hap2_col) if you have a custom pairing
    discovery_sample_col="population",
    outdir=Path("simulation_realistic/msprime_A_B_arraylike")
)

# Outputs on disk:
# - genotypes_array_diploid.parquet   (variants x individuals, values in {0,1,2})
# - sites_array.parquet
# - samples_individuals.parquet       (one row per individual incl. 'population' if available)
# - ascertainment_index.parquet       (indices back to the original variants)
# - manifest.json


In [15]:
G_chip

,0,1,2,3,4,5,6,7,8,9,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
0,1,1,0,0,0,1,1,0,0,1,...,0,0,0,1,1,0,1,1,0,1
1,1,1,1,1,1,1,0,1,1,1,...,1,1,1,0,1,1,0,0,1,1
2,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,0,1,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,1,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7044,0,1,1,0,1,0,1,0,0,0,...,1,0,1,0,0,1,1,0,0,1
7045,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7046,1,0,0,1,0,1,1,0,0,1,...,0,1,0,1,1,1,1,0,0,1
7047,1,1,0,0,1,0,1,1,0,1,...,0,1,0,1,0,1,1,0,0,1


In [16]:
sites_chip

,pos,id,alleles
0,1888,sim_10,"[C, G]"
1,11402,sim_68,"[A, T]"
2,24412,sim_157,"[A, T]"
3,31792,sim_204,"[A, T]"
4,40829,sim_263,"[A, G]"
...,...,...,...
7044,49966280,sim_355731,"[T, C]"
7045,49977762,sim_355816,"[T, C]"
7046,49984332,sim_355872,"[T, C]"
7047,49993037,sim_355929,"[G, C]"


In [17]:
samples

,sample_index,node,population_id,population
0,0,0,1,A
1,1,1,1,A
2,2,2,1,A
3,3,3,1,A
4,4,4,1,A
...,...,...,...,...
1995,1995,1995,2,B
1996,1996,1996,2,B
1997,1997,1997,2,B
1998,1998,1998,2,B


In [ ]:
outdir = Path("simulation_realistic/msprime²_A_B_arraylike")
outdir.mkdir(parents=True, exist_ok=True)
G_chip.to_parquet(outdir / "genotypes_array.parquet", index=False)
sites_chip.to_parquet(outdir / "sites_array.parquet", index=False)
samples.to_parquet(outdir / "samples.parquet", index=False)

# Also persist the ascertainment index (relative to original variants)
pd.Series(idx, name="ascertainment_index").to_frame().to_parquet(outdir / "ascertainment_index.parquet")
